# Advanced FastAPI Features - Query Parameters, Pydantic Models, and Asynchronous Tasks

## Overview
In this notebook, we will delve deeper into **FastAPI** by exploring advanced features such as **query parameter validation**, **Pydantic models** for complex data validation, and **asynchronous tasks**. We'll also cover how to handle nested data structures, filter data based on query parameters, and implement background tasks like sending emails.


## Table of Contents
1. **Query Parameter Validation**
2. **Pydantic Models for Data Validation**
3. **Nested Data Structures**
4. **Filtering Data with Query Parameters**
5. **Asynchronous Background Tasks**

## 1. Query Parameter Validation

### Definition:
**Query Parameter Validation** refers to the process of validating and constraining the input values passed in the URL query string. FastAPI provides built-in support for query parameter validation using the `Query` class, which allows you to define default values, constraints, and even regular expressions for query parameters.

### Example 1: Finding Restaurants with Query Parameters

```python
from fastapi import FastAPI, Query

app = FastAPI()

@app.get("/restaurants/")
async def find_restaurants(cuisine: str = Query(None), vegetarian: bool = Query(False)):
    """
    This endpoint allows users to search for restaurants based on cuisine and vegetarian options.
    - `cuisine`: An optional query parameter specifying the type of cuisine.
    - `vegetarian`: An optional boolean query parameter indicating whether the restaurant is vegetarian-friendly.
    """
    return {"cuisine": cuisine, "vegetarian": vegetarian}
```

#### Explanation:
- **Route**: `/restaurants/`
- **Parameters**:
  - `cuisine`: An optional query parameter that specifies the type of cuisine (e.g., Italian, Mexican).
  - `vegetarian`: An optional boolean query parameter that filters restaurants based on vegetarian options.
- **Functionality**: The function returns the provided `cuisine` and `vegetarian` values. If no values are provided, it defaults to `None` for `cuisine` and `False` for `vegetarian`.

## 2. Pydantic Models for Data Validation

### Definition:
**Pydantic Models** are used for data validation and serialization in FastAPI. They allow you to define the structure of incoming and outgoing data, enforce type constraints, and apply additional validation rules using fields like `Field`, `min_length`, `max_length`, and custom patterns. Pydantic ensures that the data conforms to the expected format before it is processed by your application.

### Example 2: Creating a User with Validation Constraints

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class User(BaseModel):
    username: str = Field(min_length=5, max_length=10, pattern="^[a-zA-Z0-9]+$")

@app.post("/create-user/")
async def create_user(user: User):
    """
    This endpoint creates a new user with specific validation constraints.
    - `username`: A string field with a minimum length of 5, maximum length of 10, and must only contain alphanumeric characters.
    """
    return {"username": user.username}
```

#### Explanation:
- **Pydantic Model**: `User` defines the structure of the incoming data with validation constraints.
  - `username`: Must be between 5 and 10 characters long and can only contain alphanumeric characters.
- **Route**: `/create-user/`
- **Functionality**: The function validates the incoming `username` using the `User` model and returns it if valid.

## 3. Nested Data Structures

### Definition:
**Nested Data Structures** refer to the use of complex data types such as lists, dictionaries, or other models within Pydantic models. This allows you to represent hierarchical or relational data in your API, such as families, organizations, or any other entities that have multiple levels of relationships.

### Example 4: Creating a Family Tree with Nested Models

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Person(BaseModel):
    name: str
    age: int

class Family(BaseModel):
    parents: list[Person]
    children: list[Person]

@app.post("/family-tree/")
async def create_family(family: Family):
    """
    This endpoint creates a family tree with nested data structures.
    - `parents`: A list of `Person` objects representing the parents.
    - `children`: A list of `Person` objects representing the children.
    """
    return family
```

#### Explanation:
- **Pydantic Models**:
  - `Person`: Represents an individual with `name` and `age`.
  - `Family`: Represents a family with lists of `parents` and `children`, both of which are lists of `Person` objects.
- **Route**: `/family-tree/`
- **Functionality**: The function validates the incoming family data and returns it if valid.

## 4. Filtering Data with Query Parameters

### Definition:
**Filtering Data with Query Parameters** allows users to retrieve specific subsets of data from an API by passing optional query parameters. These parameters can be used to filter, sort, or paginate the data returned by the API. FastAPI makes it easy to implement filtering by allowing you to define optional query parameters and apply logic to filter the data accordingly.

### Example 5: Viewing Events with Optional Filtering

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field
from datetime import date

app = FastAPI()

class Event(BaseModel):
    name: str = Field(..., min_length=1)
    event_date: date
    type: str = Field(..., min_length=1)
    location: str

events = []

@app.post("/add-event/")
async def add_event(event: Event):
    """
    This endpoint adds a new event to the events list.
    - `name`: The name of the event.
    - `event_date`: The date of the event.
    - `type`: The type of the event (e.g., conference, party).
    - `location`: The location of the event.
    """
    events.append(event.dict())
    return {"message": "Event added successfully!"}

@app.get("/view-events/")
async def view_events(event_date: date = None, type: str = None, location: str = None):
    """
    This endpoint allows users to view events with optional filtering.
    - `event_date`: Filter events by date.
    - `type`: Filter events by type.
    - `location`: Filter events by location.
    """
    filtered_events = [event for event in events if 
            (event_date is None or event['event_date'] == event_date) and
            (type is None or event['type'] == type) and
            (location is None or event['location'] == location)]
    return filtered_events
```

#### Explanation:
- **Pydantic Model**: `Event` defines the structure of the incoming data with fields for `name`, `event_date`, `type`, and `location`.
- **Routes**:
  - `/add-event/`: Adds a new event to the `events` list.
  - `/view-events/`: Returns a filtered list of events based on optional query parameters (`event_date`, `type`, `location`).
- **Functionality**: Users can add events and view them with optional filtering.

## 5. Asynchronous Background Tasks

### Definition:
**Asynchronous Background Tasks** are tasks that run in the background without blocking the main execution flow of your application. In FastAPI, you can use Python's `asyncio` library to perform background tasks such as sending emails, processing large datasets, or scheduling periodic tasks. These tasks run concurrently, allowing your API to remain responsive while performing long-running operations.

### Example 6: Submitting Time Capsule Items with Asynchronous Email Sending

```python
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr, Field
from datetime import datetime
import asyncio

app = FastAPI()

class TimeCapsuleItem(BaseModel):
    email: EmailStr
    message: str
    send_date: datetime = Field(..., example="2023-12-31T00:00:00")

time_capsule_storage = []

@app.post("/submit-time-capsule/")
async def submit_time_capsule(item: TimeCapsuleItem):
    """
    This endpoint allows users to submit time capsule items.
    - `email`: The email address where the message will be sent.
    - `message`: The message to be sent.
    - `send_date`: The date when the message should be sent.
    """
    time_capsule_storage.append(item)
    return {"message": "Time capsule item submitted successfully!"}

async def check_and_send_emails():
    """
    This asynchronous function checks the current date and sends emails for time capsule items whose send_date matches today's date.
    """
    while True:
        current_date = datetime.now()
        for item in time_capsule_storage:
            if item.send_date.date() == current_date.date():               
                print(f"Sending email to {item.email}: {item.message}")
        await asyncio.sleep(86400)  # Sleep for 24 hours (86400 seconds)
```

#### Explanation:
- **Pydantic Model**: `TimeCapsuleItem` defines the structure of the incoming data with fields for `email`, `message`, and `send_date`.
- **Route**: `/submit-time-capsule/`
- **Functionality**:
  - Users can submit time capsule items, which are stored in `time_capsule_storage`.
  - The `check_and_send_emails` function runs asynchronously, checking every 24 hours to see if any messages need to be sent based on the `send_date`.